In [1]:
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split

In [2]:
def calcola_NN50_v1(rr_list):
    NN50_v1_count=0
    for i in range(0,len(rr_list)-1): 
        if(abs(rr_list[i]-rr_list[i+1])-0.050>=0):
            NN50_v1_count+=1
    return NN50_v1_count

In [3]:
def calcola_NN50_v2(list):
    NN50_v2_count=0
    for i in range(1,len(list)):
        if(abs(list[i]-list[i-1])-0.050>=0):
            NN50_v2_count+=1
    return NN50_v2_count

In [4]:
def serial_corr(y1, lag=1):
    y2=np.roll(y1,lag)
    m=statistics.mean(y1)
    num=0
    denom=0
#    print(np.corrcoef(y1,y2)[0,1])
    for i in range(0,len(y1)):
        num+=(y1[i]-m)*(y2[i]-m)
        denom+=(y1[i]-m)**2
    return num/denom

In [5]:
def calcola_NEP(lista):
    sommatoria=0
    if(len(lista)<=2):
        return(np.nan)
    for i in range(1,len(lista)-1):
        var=(lista[i]-lista[i-1])*(lista[i+1]-lista[i])
        sommatoria+=(1-np.heaviside(var,0.5))
    return sommatoria/(len(lista)-2)

In [6]:
fileName=['a01','a02','a03','a04','a05','a06','a07','a08','a09','a10','a11','a12','a13','a14','a15','a16','a17','a18','a19','a20',
         'b01','b02','b03','b04','b05','c01','c02','c03','c04','c05','c06','c07','c08','c09','c10',
         'x01','x02','x03','x04','x05','x06','x07','x08','x09','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20']
mypath='C:/Users/sara/Documents/5anno/TESI/DataBase_PhysioNet/www.physionet.org/physiobank/database/apnea-ecg/'


In [7]:
df_features_list=[]
for index in range(0,34):
#index=0
    recordname=mypath+fileName[index]
    print(index)
    record = wfdb.rdsamp(recordname)
    dfECG=pd.DataFrame()
    ecg=record[0]
    Fs_ecg=record[1]['fs']
    intervallo=len(ecg)/(len(ecg)*Fs_ecg)
    time_ecg=[]
    for i in range(0,len(ecg)):
        time_ecg.append(intervallo*i)
    dfECG['time']=time_ecg
    dfECG['ecg']=ecg

    # valori label APNEA
    dfAPNEA=pd.DataFrame()
    ann = wfdb.rdann(recordname, extension="apn")
    time_apn=ann.sample/100
    apn=ann.symbol

    dfAPNEA['time']=time_apn
    dfAPNEA['apnea']=apn

    # valori QRS
    dfQRS=pd.DataFrame()
    ann = wfdb.rdann(recordname, extension="qrs")
    time_qrs=ann.sample/100
    dfQRS['time']=time_qrs


    #leggo RR da csv
    dfQRS = pd.read_pickle( mypath+'RR-EDR/'+fileName[index]+'QRS.pkl')
    RR_intervals=dfQRS['rr']

    #RIDIMENSIONO I DB

    maxTimeQRS=dfQRS['time'][len(dfQRS)-1]
    maxTimeECG=dfECG['time'][len(dfECG)-1]
    maxTimeAPN=dfAPNEA['time'][len(dfAPNEA)-1]
    if(maxTimeAPN<maxTimeECG and maxTimeAPN<maxTimeQRS):
        dfECG=dfECG[dfECG['time']<=(time_apn[len(time_apn)-1]+60)]
        dfQRS=dfQRS[dfQRS['time']<=(time_apn[len(time_apn)-1]+60)]
    elif(maxTimeECG<maxTimeAPN and maxTimeECG<maxTimeQRS):
        dfAPNEA=dfAPNEA[dfAPNEA['time']<=(time_ecg[len(time_ecg)-1])]
        dfQRS=dfQRS[dfQRS['time']<=(time_ecg[len(time_ecg)-1])]
    elif(maxTimeQRS<maxTimeAPN and maxTimeQRS<maxTimeECG):
        dfAPNEA=dfAPNEA[dfAPNEA['time']+60<=(time_qrs[len(time_qrs)-1])]
        dfECG=dfECG[dfECG['time']<=(time_qrs[len(time_qrs)-1])]

    #creo RR_intervals
    '''RR_intervals=[]
    for i in range (1,len(dfQRS)):
        RR_intervals.append(dfQRS['time'][i]-dfQRS['time'][i-1])
    RR_intervals.append(np.nan)
    dfQRS['rr']=RR_intervals'''



    #divido in epoch di 60 s
    lunghezza_intervalli=60*Fs_ecg
    df=pd.DataFrame();
    startTime=[]
    stopTime=[]
    startRRIndex=[]
    stopRRIndex=[]

    numIntervalli=len(dfECG)/lunghezza_intervalli
    if(len(ecg)%lunghezza_intervalli!=0):
        numIntervalli+=1
   # print(numIntervalli)
    for i in range(0,math.floor(numIntervalli)-1):
       # mediaECG.append((ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]).mean())
        startTime.append(dfECG['time'][i*lunghezza_intervalli])
        stopTime.append(dfECG['time'][i*lunghezza_intervalli+lunghezza_intervalli-1])
       # devECG.append(np.std(ecg[i*lunghezza_intervalli:i*lunghezza_intervalli+lunghezza_intervalli-1]))

    df['startTime']=startTime  
    df['stopTime']=stopTime  
    df['time_apn']=dfAPNEA['time']
    df['label']=dfAPNEA['apnea']

    k=0
    for i in range (0, len(df)):
            if(k<len(RR_intervals)):
                startRRIndex.append(k)
                k=k+1
                while( k<len(dfQRS) and df['stopTime'][i]>=dfQRS['time'][k] ):
                    k=k+1
                stopRRIndex.append(k-1)
                #print(k-1)
           # else:
              #  startRRIndex.append(np.nan)
               # stopRRIndex.append(np.nan)
    df['startRRIndex']=startRRIndex
    df['stopRRIndex']=stopRRIndex



    # calcolo interbeat differentials rd[i]=rr[i+1]−rr[i]
    rd_intervals=[]
    rd_intervals_2=[]
    for i in range(0,len(RR_intervals)-1):
        rd_intervals.append(RR_intervals[i+1]-RR_intervals[i])
    rd_intervals_2=np.power(rd_intervals,2)

    #label
    label=[]
    for i in range(0,len(dfAPNEA)):
        if(dfAPNEA['apnea'][i]=='N'):
            label.append(0)
        else:
            label.append(1)


    #CALCOLO LE FEATURES
    df_features=pd.DataFrame()
    mediaRR=[]
    stdRR=[]
    NN50_v1=[]
    NN50_v2=[]
    pNN50_v1=[]
    pNN50_v2=[]
    durata=[]
    mean_rd=[]
    std_rd=[]
    RMSDD=[]
    serialCC_1=[]
    serialCC_2=[]
    serialCC_3=[]
    serialCC_4=[]
    serialCC_5=[]
    NEP=[]
    iqr=[]
    db=[]
    ffts=[]
    for i in range(0,len(df)):
        durata.append(df['stopRRIndex'][i]-df['startRRIndex'][i])
        if(durata[i]<2):
            mediaRR.append(np.nan)
            stdRR.append(np.nan)
            NN50_v1.append(np.nan)
            NN50_v2.append(np.nan)
            pNN50_v1.append(np.nan)
            pNN50_v2.append(np.nan)
            mean_rd.append(np.nan)
            std_rd.append(np.nan)
            RMSDD.append(np.nan)
            serialCC_1.append(np.nan)
            serialCC_2.append(np.nan)
            serialCC_3.append(np.nan)
            serialCC_4.append(np.nan)
            serialCC_5.append(np.nan)
            NEP.append(np.nan)
            iqr.append(np.nan)
            ffts.append(np.nan)

        else:
            intervalloRR=np.array(RR_intervals[df['startRRIndex'][i]:df['stopRRIndex'][i]])
            mediaRR.append(statistics.mean(intervalloRR))
            stdRR.append(statistics.stdev(intervalloRR))
            NN50_v1.append(calcola_NN50_v1(intervalloRR))
            NN50_v2.append(calcola_NN50_v2(intervalloRR))
            pNN50_v1.append(NN50_v1[i]/durata[i])
            pNN50_v2.append(NN50_v2[i]/durata[i])
            mean_rd.append(statistics.mean(rd_intervals[df['startRRIndex'][i]:df['stopRRIndex'][i]]))
            std_rd.append(statistics.stdev(rd_intervals[df['startRRIndex'][i]:df['stopRRIndex'][i]]))
            RMSDD.append(math.sqrt(statistics.mean(rd_intervals_2[df['startRRIndex'][i]:df['stopRRIndex'][i]])))
            serialCC_1.append(serial_corr(intervalloRR,1))
            serialCC_2.append(serial_corr(intervalloRR,2))
            serialCC_3.append(serial_corr(intervalloRR,3))
            serialCC_4.append(serial_corr(intervalloRR,4))
            serialCC_5.append(serial_corr(intervalloRR,5))
            NEP.append(calcola_NEP(intervalloRR))
            q75, q25 = np.percentile(intervalloRR, [75 ,25])
            iqr.append(q75 - q25)
            ffts.append(fft(intervalloRR))

        db.append(index)

    df_features['db']=db
    df_features['mediaRR']=mediaRR
    df_features['stdRR']=stdRR
    df_features['NN50_v1']=NN50_v1
    df_features['NN50_v2']=NN50_v2
    df_features['pNN50_v1']=pNN50_v1
    df_features['pNN50_v2']=pNN50_v2
    df_features['mean_rd']=mean_rd
    df_features['std_rd']=std_rd
    df_features['RMSDD']=RMSDD
    df_features['serialCC_1']=serialCC_1
    df_features['serialCC_2']=serialCC_2
    df_features['serialCC_3']=serialCC_3
    df_features['serialCC_4']=serialCC_4
    df_features['serialCC_5']=serialCC_5
    df_features['NEP']=NEP
    df_features['fft']=ffts
    if(len(label)==len(NEP)+1):
        label=label[:-1]
    df_features['label']=label

    df_features=df_features.dropna() 
    df_features_list.append(df_features)

0
490.00016666666664
1
529.0001666666667
2
520.0001666666667
3
493.00016666666664
4
454.25
5
510.6666666666667
6
511.0833333333333
7
501.4166666666667
8
495.00016666666664
9
517.9166666666666
10
466.00016666666664
11
577.0
12
495.25
13
510.00016666666664
14
510.3333333333333
15
482.3333333333333
16
485.0833333333333
17
490.00016666666664
18
502.0
19
510.0
20
487.1666666666667
21
518.0001666666667
22
441.5
23
429.8333333333333
24
433.25
25
484.1666666666667
26
502.1666666666667
27
454.3333333333333
28
482.1666666666667
29
466.5
30
468.3333333333333
31
429.1666666666667
32
513.0001666666667
33
468.5


In [9]:
df_tot=pd.DataFrame()
for i in range(0,len(df_features_list)):
    df_tot=df_tot.append(df_features_list[i],ignore_index=True)

In [11]:
df_tot.to_pickle(mypath+'Features/Features.pkl')

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['0','1'], normalize=True,
                      title='Normalized confusion matrix')